In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
import b3d.chisight.shared.particle_system as ps
import genjax
from genjax import Pytree
import jax
from b3d import Pose
import b3d

In [8]:
genjax.pretty()

In [9]:
key = jax.random.PRNGKey(125)

In [11]:
key = jax.random.PRNGKey(125)

max_num_timesteps = Pytree.const(8)
num_timesteps = 4
num_particles = Pytree.const(5)
num_clusters = Pytree.const(3)
relative_particle_poses_prior_params = (Pose.identity(), .5, 0.25)
initial_object_poses_prior_params = (Pose.identity(), 2., 0.5)
camera_pose_prior_params = (Pose.identity(), 0.1, 0.1)
instrinsics = Pytree.const(b3d.camera.Intrinsics(120, 100, 50., 50., 50., 50., 0.001, 16.))
sigma_obs = 0.2

trace = ps.sparse_gps_model.simulate(key, (
    (
        max_num_timesteps, # const object
        num_timesteps,
        num_particles, # const object
        num_clusters, # const object
        relative_particle_poses_prior_params,
        initial_object_poses_prior_params,
        camera_pose_prior_params
    ),
    (instrinsics, sigma_obs)
))
trace

StaticTrace(...)

In [14]:
trace.get_choices()

StaticChm(...)

In [66]:
trace.get_retval()

({'absolute_particle_poses': Pose(position=Array([[[-0.9528756 , -0.40530294,  1.3692751 ],
          [-0.97734797,  0.31867456,  1.5139078 ],
          [-1.0789539 ,  0.06712027,  2.086363  ],
          [-1.1142604 ,  0.20091313,  1.4898124 ],
          [-1.0994865 ,  0.30513468,  1.2427154 ]],
  
         [[-0.9533268 , -0.36046904,  1.4072207 ],
          [-0.93427503,  0.37267604,  1.4955177 ],
          [-1.0726151 ,  0.17369497,  2.0808184 ],
          [-1.1837189 ,  0.29162672,  1.5091193 ],
          [-1.1654754 ,  0.35216707,  1.248084  ]],
  
         [[-0.9424262 , -0.2638655 ,  1.414935  ],
          [-0.89977545,  0.46134415,  1.5487738 ],
          [-0.9974192 ,  0.22648253,  2.1289582 ],
          [-1.1914495 ,  0.35473692,  1.435249  ],
          [-1.2087078 ,  0.4249313 ,  1.176575  ]],
  
         [[-1.0331012 , -0.24576056,  1.519422  ],
          [-1.038546  ,  0.4886248 ,  1.5988553 ],
          [-0.98764724,  0.29222953,  2.1989775 ],
          [-1.2045099 ,  0.39532572,  1.4539119 ],
          [-1.1509674 ,  0.47284877,  1.202395  ]]], dtype=float32), quaternion=Array([[[ 0.27696636,  0.6599261 , -0.24936783,  0.6523825 ],
          [ 0.39394188,  0.25852662, -0.37828475,  0.7967901 ],
          [ 0.2897191 ,  0.44323856, -0.25224078,  0.8099241 ],
          [ 0.6574079 , -0.2650006 , -0.17093046,  0.6843774 ],
          [ 0.62373984,  0.3165029 , -0.43340516,  0.56827325]],
  
         [[ 0.27781165,  0.6279244 , -0.29062846,  0.6663834 ],
          [ 0.37830287,  0.21557198, -0.40642142,  0.8032667 ],
          [ 0.2774114 ,  0.40804636, -0.29008088,  0.81999654],
          [ 0.6147988 , -0.25286785, -0.07900947,  0.7428579 ],
          [ 0.5590363 ,  0.30208394, -0.40100873,  0.6598603 ]],
  
         [[ 0.29451355,  0.6595539 , -0.28721902,  0.62909126],
          [ 0.3906492 ,  0.25581977, -0.42168507,  0.77725875],
          [ 0.29691976,  0.44590053, -0.29545087,  0.7910248 ],
          [ 0.6233946 , -0.19749746, -0.11978933,  0.74701035],
          [ 0.5413242 ,  0.35620606, -0.42754167,  0.63031226]],
  
         [[ 0.20804022,  0.7264471 , -0.3088021 ,  0.57761157],
          [ 0.30348578,  0.34168607, -0.43156454,  0.77775264],
          [ 0.21433239,  0.53160185, -0.30207723,  0.76171553],
          [ 0.6401472 , -0.30739442, -0.06090964,  0.7014345 ],
          [ 0.6116481 ,  0.2636555 , -0.3645146 ,  0.65077   ]]],      dtype=float32)),
  'camera_pose': Pose(position=Array([[ 0.04087765, -0.05517682, -0.04719735],
         [ 0.05087424,  0.02930874, -0.02247635],
         [ 0.00483565,  0.02906773,  0.05237842],
         [ 0.02782554,  0.08033713,  0.05817646]], dtype=float32), quaternion=Array([[ 0.04856758,  0.02522386,  0.0471243 ,  0.9973887 ],
         [ 0.11198938,  0.11692657, -0.22310896,  0.9612538 ],
         [ 0.05439586,  0.35174382, -0.10001067,  0.9291476 ],
         [-0.00813139,  0.2683416 , -0.27706167,  0.92258525]],      dtype=float32)),
  'object_poses': Pose(position=Array([[[-0.9563089 , -0.13892718,  1.7104663 ],
          [-1.4194523 ,  0.01832429,  1.30324   ],
          [-0.9551179 , -0.5426709 ,  1.3210348 ]],
  
         [[-0.95090264, -0.06855781,  1.7268475 ],
          [-1.4479473 ,  0.03741589,  1.3452585 ],
          [-0.9150941 , -0.5290287 ,  1.4024628 ]],
  
         [[-0.9087966 ,  0.00610334,  1.7516325 ],
          [-1.4725752 ,  0.10269226,  1.2983845 ],
          [-0.9490257 , -0.5175361 ,  1.3942097 ]],
  
         [[-0.957759  ,  0.0516827 ,  1.8247586 ],
          [-1.452701  ,  0.16324165,  1.2398366 ],
          [-1.0043097 , -0.5563303 ,  1.3717915 ]]], dtype=float32), quaternion=Array([[[ 0.3902846 ,  0.4601611 , -0.04644015,  0.79609865],
          [ 0.6504725 ,  0.11579459, -0.17413272,  0.73017466],
          [ 0.31395438, -0.3877612 , -0.0230728 ,  0.866338  ]],
  
         [[ 0.3750289 ,  0.4298328 , -0.08287834,  0.8171465 ],
          [ 0.57738924,  0.12317934, -0.11343634,  0.7991125 ],
          [ 0.2781669 , -0.3508015 ,  0.06146897,

In [74]:
trace.get_retval()[0]["vis_mask"]

# jax.Array int32(4, 5) [≥0, ≤1] zero:9 nonzero:11
  Array([[1, 1, 1, 1, 0],
         [0, 0, 0, 0, 1],
         [1, 0, 1, 0, 1],
         [0, 1, 1, 0, 1]], dtype=int32)

In [31]:
tr = ps.initial_particle_system_state.simulate(key, (
    num_particles, num_clusters, relative_particle_poses_prior_params, initial_object_poses_prior_params, camera_pose_prior_params
))

In [33]:
tr.get_choices()

XorChm(...)

In [36]:
(state0, init_retval) = tr.get_retval()
init_retval

{'absolute_particle_poses': Pose(position=Array([[ 0.40394464,  0.49178421,  0.83125967],
        [ 1.1557399 ,  0.1073809 , -1.0004435 ],
        [ 0.7466202 , -0.11125076, -1.0334555 ],
        [ 0.42588845,  0.15040515,  0.7240628 ],
        [-0.08075588,  0.03178127,  0.9834795 ]], dtype=float32), quaternion=Array([[ 0.31261247,  0.3264106 ,  0.45340466,  0.7682147 ],
        [ 0.04813731, -0.29024395, -0.20838895,  0.93274605],
        [ 0.12582801, -0.71169275, -0.09951743,  0.6839277 ],
        [-0.09185741,  0.30670545,  0.40737277,  0.855302  ],
        [ 0.03260923,  0.1007082 ,  0.2827858 ,  0.9533241 ]],      dtype=float32)),
 'camera_pose': Pose(position=Array([-0.00054856,  0.0540217 , -0.05834808], dtype=float32), quaternion=Array([ 0.06375018, -0.02930492, -0.1331465 ,  0.98860973], dtype=float32)),
 'object_poses': Pose(position=Array([[-0.8351105 ,  1.3851395 ,  0.63394755],
        [ 0.9989229 , -0.10820086, -1.0928634 ],
        [-0.02473093,  0.27078107,  0.85259247]], dtype=float32), quaternion=Array([[ 0.05015931, -0.3799355 ,  0.16383284,  0.909006  ],
        [ 0.03074602, -0.5201218 ,  0.08101467,  0.8496851 ],
        [-0.00533034,  0.2970283 ,  0.3590314 ,  0.88478374]],      dtype=float32)),
 'relative_particle_poses': Pose(position=Array([[ 0.39352402, -0.1140423 ,  0.25532994],
        [ 0.17484564,  0.18277925, -0.12501697],
        [-0.05999853,  0.03789828,  0.24931735],
        [ 0.2468853 , -0.40325543,  0.10247313],
        [-0.25207934, -0.11491695,  0.02532204]], dtype=float32), quaternion=Array([[ 0.2632069 , -0.05403322,  0.21994649,  0.9377772 ],
        [-0.11967833,  0.22821863, -0.2687445 ,  0.92810005],
        [-0.02353261, -0.26224265, -0.18353081,  0.94709635],
        [-0.08759926,  0.04812734,  0.02770715,  0.9946067 ],
        [-0.01390432, -0.2072743 , -0.0818463 ,  0.9747539 ]],      dtype=float32)),
 'vis_mask': <jax.Array([0, 0, 1, 0, 0], dtype=int32)>}

In [51]:
scantr = ps.particle_system_state_step.scan(n=(num_timesteps.const - 1)).simulate(
    key, (state0, None)
)

In [52]:
scantr.get_choices()

IdxChm(...)

In [53]:
(final_state, scan_retval) = scantr.get_retval()
scan_retval

{'absolute_particle_poses': Pose(position=Array([[[ 0.36036593,  0.4931063 ,  0.87947124],
         [ 1.1646038 ,  0.05811405, -1.1079048 ],
         [ 0.73829836, -0.12721162, -1.094248  ],
         [ 0.38899153,  0.15259093,  0.77110964],
         [-0.15376458,  0.04543683,  0.94996977]],
 
        [[ 0.37076294,  0.3681712 ,  0.90084624],
         [ 1.196027  ,  0.07400203, -1.0740566 ],
         [ 0.77749217, -0.12785229, -1.0928155 ],
         [ 0.3501452 ,  0.05677184,  0.72444403],
         [-0.16809943, -0.05037972,  0.9652722 ]],
 
        [[ 0.38736004,  0.3257344 ,  0.9398421 ],
         [ 1.2504895 ,  0.02589861, -1.0647875 ],
         [ 0.8366114 , -0.15606686, -1.1737005 ],
         [ 0.29607272,  0.02570063,  0.7661794 ],
         [-0.24314705,  0.06024784,  0.9809019 ]]], dtype=float32), quaternion=Array([[[ 0.28782642,  0.2602054 ,  0.46717957,  0.7944762 ],
         [-0.04225804, -0.291873  , -0.22387134,  0.9289274 ],
         [ 0.03933909, -0.7102853 , -0.07273035,  0.6990405 ],
         [-0.11164563,  0.23999774,  0.38960102,  0.8821267 ],
         [ 0.0196385 ,  0.02642748,  0.27185038,  0.96177614]],
 
        [[ 0.41381675,  0.26070198,  0.43473133,  0.75617373],
         [ 0.06674349, -0.25198773, -0.23636203,  0.93604517],
         [ 0.13210969, -0.6864102 , -0.13979195,  0.70131755],
         [ 0.0200845 ,  0.2717025 ,  0.3836268 ,  0.8823859 ],
         [ 0.14041534,  0.07309642,  0.23153813,  0.9598597 ]],
 
        [[ 0.42608923,  0.19002779,  0.34147373,  0.81592476],
         [ 0.09314049, -0.3472136 , -0.25124246,  0.8986906 ],
         [ 0.13637125, -0.75708413, -0.14683306,  0.6218252 ],
         [ 0.03734541,  0.2480718 ,  0.26456118,  0.9311676 ],
         [ 0.13413665,  0.03236134,  0.10862075,  0.9844601 ]]],      dtype=float32)),
 'camera_pose': Pose(position=Array([[-0.05851199,  0.08131988, -0.12015802],
        [-0.03376154,  0.10507603, -0.20219323],
        [-0.02877734,  0.0587935 , -0.276977  ]], dtype=float32), quaternion=Array([[ 0.10911258,  0.18976991, -0.02298202,  0.9754762 ],
        [ 0.08588515,  0.0251347 , -0.07712492,  0.9929974 ],
        [ 0.20970307,  0.17755964,  0.11609443,  0.95447326]],      dtype=float32)),
 'object_poses': Pose(position=Array([[[-0.87120515,  1.3736898 ,  0.6607467 ],
         [ 0.987428  , -0.15515342, -1.1601944 ],
         [-0.07241641,  0.28432354,  0.832894  ]],
 
        [[-0.88771987,  1.4047782 ,  0.7261908 ],
         [ 1.0251832 , -0.12936762, -1.1692494 ],
         [-0.08224714,  0.2019974 ,  0.88573414]],
 
        [[-0.9435097 ,  1.4065702 ,  0.74644756],
         [ 1.0944561 , -0.1681515 , -1.1974776 ],
         [-0.0918551 ,  0.28034472,  0.9030608 ]]], dtype=float32), quaternion=Array([[[ 0.02159333, -0.4294057 ,  0.25887853,  0.86494297],
         [-0.05757808, -0.49832183,  0.08644408,  0.8607483 ],
         [-0.02166905,  0.22728433,  0.34740812,  0.90949434]],
 
        [[-0.00793305, -0.43463644,  0.34698018,  0.8310433 ],
         [ 0.05230644, -0.49371713,  0.04711475,  0.8667686 ],
         [ 0.10820736,  0.26193216,  0.33234182,  0.8995731 ]],
 
        [[ 0.08902398, -0.46995282,  0.35609448,  0.8027551 ],
         [ 0.04334753, -0.5824462 ,  0.02863756,  0.8112074 ],
         [ 0.12457285,  0.22612935,  0.21380606,  0.94214344]]],      dtype=float32)),
 'relative_particle_poses': Pose(position=Array([[[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.0378

In [54]:
for key in scan_retval.keys():
    print(key, scan_retval[key].shape, init_retval[key].shape)

absolute_particle_poses (3, 5) (5,)
camera_pose (3,) ()
object_poses (3, 3) (3,)
relative_particle_poses (3, 5) (5,)
vis_mask (3, 3) (5,)


In [57]:
scan_retval["vis_mask"]

# jax.Array int32(3, 3) [≥0, ≤1] zero:7 nonzero:2
  Array([[1, 0, 0],
         [0, 0, 0],
         [0, 0, 1]], dtype=int32)

In [58]:
init_retval["vis_mask"]

<jax.Array([0, 0, 1, 0, 0], dtype=int32)>

In [ ]:
trace

In [ ]:
from b3d.chisight.sparse.pose_utils import uniform_pose_in_ball
posetr = uniform_pose_in_ball.simulate(key, (Pose.identity(), 0.1, 0.2))

In [ ]:
uniform_pose_in_ball.sample(key, (Pose.identity(), 0.1, 0.2))

In [39]:
(final_state, scan_retval) = scantr.get_retval()
scan_retval

{'absolute_particle_poses': Pose(position=Array([[[ 0.36036593,  0.4931063 ,  0.87947124],
         [ 1.1646038 ,  0.05811405, -1.1079048 ],
         [ 0.73829836, -0.12721162, -1.094248  ],
         [ 0.38899153,  0.15259093,  0.77110964],
         [-0.15376458,  0.04543683,  0.94996977]],
 
        [[ 0.37076294,  0.3681712 ,  0.90084624],
         [ 1.196027  ,  0.07400203, -1.0740566 ],
         [ 0.77749217, -0.12785229, -1.0928155 ],
         [ 0.3501452 ,  0.05677184,  0.72444403],
         [-0.16809943, -0.05037972,  0.9652722 ]],
 
        [[ 0.38736004,  0.3257344 ,  0.9398421 ],
         [ 1.2504895 ,  0.02589861, -1.0647875 ],
         [ 0.8366114 , -0.15606686, -1.1737005 ],
         [ 0.29607272,  0.02570063,  0.7661794 ],
         [-0.24314705,  0.06024784,  0.9809019 ]]], dtype=float32), quaternion=Array([[[ 0.28782642,  0.2602054 ,  0.46717957,  0.7944762 ],
         [-0.04225804, -0.291873  , -0.22387134,  0.9289274 ],
         [ 0.03933909, -0.7102853 , -0.07273035,  0.6990405 ],
         [-0.11164563,  0.23999774,  0.38960102,  0.8821267 ],
         [ 0.0196385 ,  0.02642748,  0.27185038,  0.96177614]],
 
        [[ 0.41381675,  0.26070198,  0.43473133,  0.75617373],
         [ 0.06674349, -0.25198773, -0.23636203,  0.93604517],
         [ 0.13210969, -0.6864102 , -0.13979195,  0.70131755],
         [ 0.0200845 ,  0.2717025 ,  0.3836268 ,  0.8823859 ],
         [ 0.14041534,  0.07309642,  0.23153813,  0.9598597 ]],
 
        [[ 0.42608923,  0.19002779,  0.34147373,  0.81592476],
         [ 0.09314049, -0.3472136 , -0.25124246,  0.8986906 ],
         [ 0.13637125, -0.75708413, -0.14683306,  0.6218252 ],
         [ 0.03734541,  0.2480718 ,  0.26456118,  0.9311676 ],
         [ 0.13413665,  0.03236134,  0.10862075,  0.9844601 ]]],      dtype=float32)),
 'camera_pose': Pose(position=Array([[-0.05851199,  0.08131988, -0.12015802],
        [-0.03376154,  0.10507603, -0.20219323],
        [-0.02877734,  0.0587935 , -0.276977  ]], dtype=float32), quaternion=Array([[ 0.10911258,  0.18976991, -0.02298202,  0.9754762 ],
        [ 0.08588515,  0.0251347 , -0.07712492,  0.9929974 ],
        [ 0.20970307,  0.17755964,  0.11609443,  0.95447326]],      dtype=float32)),
 'object_poses': Pose(position=Array([[[-0.87120515,  1.3736898 ,  0.6607467 ],
         [ 0.987428  , -0.15515342, -1.1601944 ],
         [-0.07241641,  0.28432354,  0.832894  ]],
 
        [[-0.88771987,  1.4047782 ,  0.7261908 ],
         [ 1.0251832 , -0.12936762, -1.1692494 ],
         [-0.08224714,  0.2019974 ,  0.88573414]],
 
        [[-0.9435097 ,  1.4065702 ,  0.74644756],
         [ 1.0944561 , -0.1681515 , -1.1974776 ],
         [-0.0918551 ,  0.28034472,  0.9030608 ]]], dtype=float32), quaternion=Array([[[ 0.02159333, -0.4294057 ,  0.25887853,  0.86494297],
         [-0.05757808, -0.49832183,  0.08644408,  0.8607483 ],
         [-0.02166905,  0.22728433,  0.34740812,  0.90949434]],
 
        [[-0.00793305, -0.43463644,  0.34698018,  0.8310433 ],
         [ 0.05230644, -0.49371713,  0.04711475,  0.8667686 ],
         [ 0.10820736,  0.26193216,  0.33234182,  0.8995731 ]],
 
        [[ 0.08902398, -0.46995282,  0.35609448,  0.8027551 ],
         [ 0.04334753, -0.5824462 ,  0.02863756,  0.8112074 ],
         [ 0.12457285,  0.22612935,  0.21380606,  0.94214344]]],      dtype=float32)),
 'relative_particle_poses': Pose(position=Array([[[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.03789828,  0.24931735],
         [ 0.2468853 , -0.40325543,  0.10247313],
         [-0.25207934, -0.11491695,  0.02532204]],
 
        [[ 0.39352402, -0.1140423 ,  0.25532994],
         [ 0.17484564,  0.18277925, -0.12501697],
         [-0.05999853,  0.0378

In [40]:
for key in scan_retval.keys():
    print(key, scan_retval[key].shape, init_retval[key].shape)

absolute_particle_poses (3, 5) (5,)
camera_pose (3,) ()
object_poses (3, 3) (3,)
relative_particle_poses (3, 5) (5,)
vis_mask (3, 3) (5,)


In [50]:
from b3d.chisight.sparse.pose_utils import uniform_pose_in_ball
posetr = uniform_pose_in_ball.simulate(key, (Pose.identity(), 0.1, 0.2))
posetr.get_retval()

Pose(position=Array([-0.01156442, -0.05576068, -0.04115251], dtype=float32), quaternion=Array([-0.12245512, -0.02679094,  0.07691857,  0.98912615], dtype=float32))

In [11]:
trace.get_choices()

EmptyChm(...)